# Load medical images

This notebook introduces how to easily load different formats of medical images in MONAI and execute many additional operations.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Project-MONAI/tutorials/blob/master/modules/load_medical_images.ipynb)

## Setup environment

In [3]:
!python -c "import monai" || pip install -q monai[itk, pillow]

## Setup imports

In [ ]:
# Copyright 2020 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Copyright 2020 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import os
import shutil
import numpy as np
import itk
from PIL import Image
import tempfile
from monai.data import ITKReader, PILReader
from monai.transforms import (
    LoadImage, LoadImaged, AddChanneld,
    Resized, ToTensord, Compose
)
from monai.config import print_config

print_config()

## Load Nifti image with default image reader
MONAI automatically choose readers based on the supported suffixes and in below order:
- User specified reader at runtime when call this loader.
- Registered readers from the latest to the first in list.
- Default readers: (nii, nii.gz -> NibabelReader), (png, jpg, bmp -> PILReader),
  (npz, npy -> NumpyReader), (others -> ITKReader).

In [ ]:
# generate 3D test images
tempdir = tempfile.mkdtemp()
test_image = np.random.rand(64, 128, 96)
filename = os.path.join(tempdir, "test_image.nii.gz")
itk_np_view = itk.image_view_from_array(test_image)
itk.imwrite(itk_np_view, filename)
data, meta = LoadImage()(filename)

print(f"image data shape:{data.shape}")
print(f"meta data:{meta}")

## Load a list of Nifti images and stack as 1 training item
Loading a list of files, stack them together and add a new dimension as first dimension.

And use the meta data of the first image to represent the stacked result.

In [ ]:
filenames = ["test_image.nii.gz", "test_image2.nii.gz", "test_image3.nii.gz"]
for i, name in enumerate(filenames):
    filenames[i] = os.path.join(tempdir, name)
    itk_np_view = itk.image_view_from_array(test_image)
    itk.imwrite(itk_np_view, filenames[i])
data, meta = LoadImage()(filenames)

print(f"image data shape:{data.shape}")
print(f"meta data:{meta}")

## Load 3D image in DICOM format

In [ ]:
filename = os.path.join(tempdir, "test_image.dcm")
dcm_image = np.random.randint(256, size=(64, 128, 96)).astype(np.uint8())
itk_np_view = itk.image_view_from_array(dcm_image)
itk.imwrite(itk_np_view, filename)
data, meta = LoadImage()(filename)

print(f"image data shape:{data.shape}")
print(f"meta data:{meta}")

## Load a list of DICOM images and stack as 1 training item
Loading a list of files, stack them together and add a new dimension as first dimension.

And use the meta data of the first image to represent the stacked result.

In [2]:
filenames = ["test_image.dcm", "test_image2.dcm", "test_image3.dcm"]
for i, name in enumerate(filenames):
    filenames[i] = os.path.join(tempdir, name)
    itk_np_view = itk.image_view_from_array(dcm_image)
    itk.imwrite(itk_np_view, filenames[i])
data, meta = LoadImage()(filenames)

print(f"image data shape:{data.shape}")
print(f"meta data:{meta}")

NameError: name 'os' is not defined

## Load 2D image in PNG format

In [ ]:
test_image = np.random.randint(0, 256, size=[128, 256])
filename = os.path.join(tempdir, "test_image.png")
Image.fromarray(test_image.astype("uint8")).save(filename)
data, meta = LoadImage()(filename)

print(f"image data shape:{data.shape}")
print(f"meta data:{meta}")

## Load image with specified image reader
And we can set additional parameters for the image readers, for example, set `c_order_axis_indexing=True` for `ITKReader`, this parameter will pass to ITK `read()` function later.

In [ ]:
loader = LoadImage()
loader.register(ITKReader())
data, meta = loader(filename)

print(f"image data shape:{data.shape}")
print(f"meta data:{meta}")

## Load image and execute additional operations
Some image readers can support additional operations after reading the image from file.

For example, we can set a converter for PILReader: `PILReader(converter=lambda image: image.convert("LA"))`.

In [ ]:
loader = LoadImage(PILReader(converter=lambda image: image.convert("LA")))
data, meta = loader(filename)

print(f"image data shape:{data.shape}")
print(f"meta data:{meta}")

## Connect `LoadImage` with other transforms
It's very easy to connect `LoadImage` transform with other transforms to construct a transform chain.

In [ ]:
transform = Compose([
    LoadImaged(keys="image"),
    AddChanneld(keys="image"),
    Resized(keys="image", spatial_size=[64, 64]),
    ToTensord("image"),
])
test_data = {"image": filename}
result = transform(test_data)
print(f"image data shape:{result['image'].shape}")
print(f"meta data:{result['image_meta_dict']}")

## Cleanup data directory

Remove directory if a temporary was used.

In [ ]:
shutil.rmtree(tempdir)